In [1]:
from scipy.stats import spearmanr

from bertscore import BERTScore
from conventional_metrics import BLEU, METEOR
from scm import SCM
from wmd import WMD
from common import Evaluator, LANGS
import pandas as pd


/Users/xstefan3/miniconda3/envs/ood/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
metrics = [
    BLEU(),
    METEOR(),
    # BERTScore(tgt_lang="en"),
    WMD(tgt_lang="en"),
    SCM(tgt_lang="en", use_tfidf=False),
    SCM(tgt_lang="en", use_tfidf=True)
]
correlations = {m.label: {} for m in metrics}
correlations["human"] = {}

for lang_pair in [pair for pair in LANGS if pair.split("-")[-1] == "en"]:
    print("Evaluating lang pair %s" % lang_pair)
    evaluator = Evaluator("data_dir", lang_pair, metrics)
    report = evaluator.evaluate()

    human_judgements = report["human"]
    for metric_label, vals in report.items():
        correlations[metric_label][lang_pair] = spearmanr(vals, human_judgements).correlation
    print(correlations)

corrs_df = pd.DataFrame(correlations)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/xstefan3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xstefan3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xstefan3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xstefan3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Evaluating lang pair cs-en


  0%|          | 0/2687 [00:00<?, ?it/s]

{'BLEU': {'cs-en': 0.00948259297595841}, 'METEOR': {'cs-en': 0.40259685151100627}, 'WMD': {'cs-en': -0.35835693780579453}, 'SCM': {'cs-en': 0.24208020921663048}, 'SCM_tfidf': {'cs-en': 0.2431252749190664}, 'human': {'cs-en': 1.0}}
Evaluating lang pair de-en


  9%|▉         | 253/2687 [02:50<2:10:26,  3.22s/it] 

In [ ]:
import seaborn as sns
sns.heatmap(corrs_df, annot=True)